In [ ]:
!pip install numpy==1.22

In [ ]:
!pip install LibRecommender==0.12.0

  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb  7 21:06:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

##  data set


### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1tZlrPeJf7geYfRRTxeRhdPU_Hq5m8tg6

Downloading...
From: https://drive.google.com/uc?id=1tZlrPeJf7geYfRRTxeRhdPU_Hq5m8tg6
To: /content/Travel_TripAdvisor_v1.zip
100% 155k/155k [00:00<00:00, 93.9MB/s]


In [ ]:
!unzip "Travel_TripAdvisor_v1.zip"  -d  "/content"

Archive:  Travel_TripAdvisor_v1.zip
replace /content/Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls  
  inflating: /content/Travel_TripAdvisor_v1/ReadMe.html  


In [ ]:
!pip install pandas openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import pandas as pd

xls = pd.ExcelFile(r"Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls") 

rating_df = xls.parse(0) #2 is the sheet number+1 thus if the file has only 1 sheet write 0 in paranthesis



In [ ]:
rating_df

,ID_USER,USER_STATE,USER_TIMEZONE,ID_HOTEL,HOTEL_CITY,HOTEL_STATE,HOTEL_TIMEZONE,Trip Type,Rating
0,45,GA,Eastern,105170,Memphis,TN,Central,3,5
1,45,GA,Eastern,223229,SanAntonio,TX,Central,3,5
2,45,GA,Eastern,258688,Albuquerque,NM,Mountain,3,5
3,45,GA,Eastern,98827,ELPaso,TX,Central,3,5
4,45,GA,Eastern,99518,SanAntonio,TX,Central,3,3
...,...,...,...,...,...,...,...,...,...
4664,65440,MI,Eastern,95715,Minneapolis,MN,Central,4,5
4665,65457,AZ,Mountain,1027019,FortWorth,TX,Central,3,3
4666,65457,AZ,Mountain,224458,Milwaukee,WI,Central,3,3
4667,65457,AZ,Mountain,223749,Columbus,OH,Eastern,3,2


In [ ]:
rating_df=rating_df.reset_index(drop=True)

In [ ]:

rating_df.rename(columns = {'ID_USER':'user'}, inplace = True)
rating_df.rename(columns = {'ID_HOTEL':'item'}, inplace = True)
rating_df.rename(columns = {'Rating':'label'}, inplace = True)

## Train

In [ ]:

import tensorflow as tf

from libreco.data import DatasetFeat, DataInfo
from libreco.data import split_by_ratio_chrono,split_by_ratio
from libreco.algorithms import DeepFM,YouTubeRanking
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
rating_df.columns

Index(['user', 'USER_STATE', 'USER_TIMEZONE', 'item', 'HOTEL_CITY',
       'HOTEL_STATE', 'HOTEL_TIMEZONE', 'Trip Type', 'label'],
      dtype='object')

In [ ]:

col_sparse= ['USER_STATE', 'USER_TIMEZONE','Trip Type']
user_col = ['USER_STATE', 'USER_TIMEZONE','Trip Type']
item_col=None
dense_col=None

In [ ]:
rating_df=rating_df[['user','item','label','USER_STATE', 'USER_TIMEZONE', 'Trip Type']]

In [ ]:
# rating_df['label']=1

In [ ]:
train, test = split_by_ratio(rating_df, test_size=0.2)

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(
        train, user_col, item_col, col_sparse, dense_col
    )

In [ ]:
test_data = DatasetFeat.build_testset(test, shuffle=False)

In [ ]:
import torch
from libreco.algorithms import (
    FM,
    WideDeep,
    DeepFM
)


    # # only do negative sampling on eval data
    # eval_data.build_negative_samples(
    #     data_info, item_gen_mode="random", num_neg=1, seed=2222
    # )



## Widedeep

In [ ]:
wd = WideDeep(
        "ranking",
        data_info,
        embed_size=16,
        n_epochs=10,
        lr={"wide": 0.01, "deep": 1e-4},
        lr_decay=False,
        reg=None,
        batch_size=256,
        num_neg=1,
        use_bn=False,
        dropout_rate=None,
        hidden_units="128,64,32",
        tf_sess_config=None,
    )
wd.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=[ "precision", "recall", "map", "ndcg"],
    )


total params: 72,848 | embedding params: 52,331 | network params: 20,517
Training start time: 2023-02-07 21:13:10


train: 100%|██████████| 17/17 [00:00<00:00, 17.92it/s]


Epoch 1 elapsed: 0.955s
	 train_loss: 0.4351


eval_rec: 100%|██████████| 411/411 [00:00<00:00, 708.63it/s]


	 eval precision@10: 0.0015
	 eval recall@10: 0.0146
	 eval map@10: 0.0026
	 eval ndcg@10: 0.0053


...

	 eval precision@10: 0.0022
	 eval recall@10: 0.0207
	 eval map@10: 0.0035
	 eval ndcg@10: 0.0075


train: 100%|██████████| 17/17 [00:00<00:00, 264.58it/s]


Epoch 10 elapsed: 0.069s
	 train_loss: -12.5001


eval_rec: 100%|██████████| 411/411 [00:00<00:00, 777.24it/s]


	 eval precision@10: 0.0015
	 eval recall@10: 0.0146
	 eval map@10: 0.0035
	 eval ndcg@10: 0.0060


In [ ]:
print( "recommendation: ",
        wd.recommend_user(
            user=65457,
            n_rec=7,
            inner_id=False,
            cold_start="average",
            user_feats=pd.Series({"USER_STATE": "GA", "USER_TIMEZONE": "Mountain", "Trip Type": 1}),
           
        ))

recommendation:  [(105455, 0.99555427), (98197, 0.9954384), (224227, 0.9953811), (596431, 0.99536884), (108149, 0.9953622), (111855, 0.9953497), (108972, 0.9953157)]


In [ ]:
print("recommendation: ", wd.recommend_user(user=1, n_rec=7))

detect unknown user: 1
recommendation:  [(105455, 0.99534494), (98197, 0.9952237), (224227, 0.9951626), (596431, 0.99515015), (108149, 0.9951439), (111855, 0.9951304), (108972, 0.99509466)]


## Youtube retrival

In [ ]:

ytb_ranking = YouTubeRanking(task="ranking", data_info=data_info, embed_size=64,
                             n_epochs=20, lr=1e-4, batch_size=256, use_bn=True,
                             hidden_units="128,128,32")
ytb_ranking.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=[ "precision", "recall", "map", "ndcg"])



Instructions for updating:
Colocations handled automatically by placer.


total params: 267,457 | embedding params: 196,449 | network params: 71,008
Training start time: 2023-02-05 11:02:36


train: 100%|██████████| 17/17 [00:00<00:00, 17.89it/s]


Epoch 1 elapsed: 0.957s
	 train_loss: 0.7701


eval_rec: 100%|██████████| 411/411 [00:00<00:00, 437.74it/s]


...

	 eval precision@10: 0.0002
	 eval recall@10: 0.0024
	 eval map@10: 0.0006
	 eval ndcg@10: 0.0010


train: 100%|██████████| 17/17 [00:00<00:00, 177.72it/s]


Epoch 20 elapsed: 0.102s
	 train_loss: -5.2983


eval_rec: 100%|██████████| 411/411 [00:00<00:00, 480.84it/s]


	 eval precision@10: 0.0005
	 eval recall@10: 0.0049
	 eval map@10: 0.0009
	 eval ndcg@10: 0.0018


In [ ]:
rating_df

,user,item,label,USER_STATE,USER_TIMEZONE,Trip Type
0,45,105170,5,GA,Eastern,3
1,45,223229,5,GA,Eastern,3
2,45,258688,5,GA,Eastern,3
3,45,98827,5,GA,Eastern,3
4,45,99518,3,GA,Eastern,3
...,...,...,...,...,...,...
4664,65440,95715,5,MI,Eastern,4
4665,65457,1027019,3,AZ,Mountain,3
4666,65457,224458,3,AZ,Mountain,3
4667,65457,223749,2,AZ,Mountain,3


In [ ]:
print( "recommendation: ",
        ytb_ranking.recommend_user(
            user=65457,
            n_rec=7,
            inner_id=False,
            cold_start="average",
            user_feats=pd.Series({"USER_STATE": "GA", "USER_TIMEZONE": "Mountain", "Trip Type": 1}),
           
        ))
    

recommendation:  [(91813, 0.94110066), (633673, 0.9405274), (107844, 0.9404271), (95707, 0.9396957), (729441, 0.9396075), (95732, 0.93904823), (1201174, 0.93902487)]


In [ ]:


# recommend 7 items for user 2211
print("recommendation(id, probability): ", ytb_ranking.recommend_user(user=1, n_rec=7))

detect unknown user: 1
recommendation(id, probability):  [(91813, 0.94102156), (633673, 0.9404539), (107844, 0.94035184), (95707, 0.93962556), (729441, 0.93954206), (95732, 0.9389738), (1201174, 0.9389589)]


## deepFM

In [ ]:
train_data.build_negative_samples(
        data_info, num_neg=1, item_gen_mode="random", seed=2020
    )
test_data.build_negative_samples(
        data_info, num_neg=1, item_gen_mode="random", seed=2222
    )

random neg item sampling elapsed: 0.006s
random neg item sampling elapsed: 0.000s


In [ ]:
deepfm = DeepFM(
        "ranking",
        data_info,
        embed_size=128,
        n_epochs=30,
        lr=1e-4,
        lr_decay=False,
        reg=None,
        batch_size=256,
        num_neg=1,
        use_bn=False,
        dropout_rate=None,
        hidden_units="128,128,32",
        tf_sess_config=None,
    )
    

total params: 571,089 | embedding params: 448,006 | network params: 123,083


In [ ]:
deepfm.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=[
            "loss",
            "balanced_accuracy",
            "roc_auc",
            "pr_auc",
            "precision",
            "recall",
            "map",
            "ndcg",
        ],
    )


Training start time: 2023-02-07 21:13:37


train: 100%|██████████| 33/33 [00:00<00:00, 95.85it/s]


Epoch 1 elapsed: 0.353s
	 train_loss: 0.6933


eval_rec: 100%|██████████| 411/411 [00:00<00:00, 611.93it/s]


	 eval log_loss: 0.6932
	 eval balanced accuracy: 0.4989
	 eval roc_auc: 0.4962
	 eval pr_auc: 0.5047
	 eval precision@10: 0.0005
	 eval recall@10: 0.0049
	 eval map@10: 0.0010
	 eval ndcg@10: 0.0019


...

	 eval log_loss: 0.8362
	 eval balanced accuracy: 0.5538
	 eval roc_auc: 0.5808
	 eval pr_auc: 0.5580
	 eval precision@10: 0.0010
	 eval recall@10: 0.0097
	 eval map@10: 0.0023
	 eval ndcg@10: 0.0040


train: 100%|██████████| 33/33 [00:00<00:00, 208.93it/s]


Epoch 30 elapsed: 0.164s
	 train_loss: 0.5752


eval_rec: 100%|██████████| 411/411 [00:00<00:00, 641.01it/s]


	 eval log_loss: 0.8436
	 eval balanced accuracy: 0.5504
	 eval roc_auc: 0.5812
	 eval pr_auc: 0.5592
	 eval precision@10: 0.0010
	 eval recall@10: 0.0097
	 eval map@10: 0.0023
	 eval ndcg@10: 0.0040


In [ ]:
print( "recommendation: ",
        deepfm.recommend_user(
            user=65457,
            n_rec=7,
            inner_id=False,
            cold_start="average",
            user_feats=pd.Series({"USER_STATE": "GA", "USER_TIMEZONE": "Mountain", "Trip Type": 1}),
           
        ))
    

recommendation:  [(1176838, 0.99383575), (98674, 0.9916346), (98827, 0.990902), (217517, 0.99076664), (570279, 0.9906881), (299922, 0.99065405), (1200984, 0.99055)]
